## cross_val_score 평가
- cross_val_score
(사용할 모델 , x값 , y값 , scroing(평가할 스코어링 방법),cv(몇번 검증할 것인가 ?),n_jobs(cpu사용자에 대한 지정-1), verbose(진행 사항 보기) )

In [1]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.datasets import load_iris
import numpy as np
import pandas as pd

In [2]:
iris_data = load_iris()
data =  iris_data.data
label = iris_data.target

In [3]:
dt_clf = DecisionTreeClassifier(random_state=111)

In [4]:
# 교차검증 진행
scores = cross_val_score(dt_clf,data,label,scoring='accuracy',cv=5)
print(np.round(np.mean(scores),3))

0.96


In [6]:
crs=cross_validate(dt_clf, data, label, scoring='accuracy',cv=5, return_train_score=True)
pd.DataFrame(crs) #시간까지 확인 가능 

,fit_time,score_time,test_score,train_score
0,0.000997,0.000999,0.966667,1.0
1,0.001982,0.001000,0.966667,1.0
2,0.002001,0.001001,0.900000,1.0
3,0.000999,0.001000,0.966667,1.0
4,0.002004,0.003033,1.000000,1.0


In [9]:
import seaborn as sns
df = sns.load_dataset('titanic')

In [10]:
df_tt = df[['pclass','fare']] # x

In [11]:
scores = cross_val_score(dt_clf,df_tt,df['survived'],scoring='f1',cv=5)
print(np.round(np.mean(scores),3)) # 이진분류면 잘 나온다

0.535


## confusion matrix 
- 정확도 , 정밀도 , 재현율 , f1스코어 , auc , roc 커브
- 이진분류는 정확도 , 정밀도 , 재현율 , f1스코어 , auc , roc 커브 분류에 디폴트로 문제없다
- 다중분류로 했을때는 어떻게 해야할까 ?

## 다중분류 진행시 평가지표에 대한 가중치 설정 방법
- _micro , _macro , _weighted 이런식으로 사용하면 된다 .
- macro
-> 모든 예측 결과에 대해서 평균을 내어서 계산하는 방법 
- micro
-> 각각의 정답에 대한 개수를 가지고 나눠서 평균값을 구한다
- weighted 
-> 가중 평균값을 구한다 , 가중치를 주고 싶은 것에 줄 수 있다 .


#### 예시 
- 4개라벨(0,1,2,3)클래스 있을떄 precision계산하면 
- 0 -> 0.5 = 1/2 , 1 -> 0.1 = 10/100 , 2 -> 0.5 = 1/2 , 3 -> 0.5 = 1/2 이면
- macro : (0.5+0.1+0.5+0.5) / 4 
- micro : (1+10+1+1) / (2+100+2+2)  

#### 그렇다면 언제 뭐를 선택 ? 
- 클래스에 대한 불균형에 따라서 선택해야 한다 (불균형일떄 micro)
- 사기탐지 같은 예시를 보면 사기가 압도적으로 적으니 불균형 상황


## leaveOneOut 교차검증
- corss_val_score 부분 cv=loo 넣는 방법
## Group 교차검증
- 그룹핑은 그룹핑 데이터를 가지고 교차검증할떄 더 좋은 성능을 보여줌
- 단순하게 교차검증을 cross 하는게 아니라 데이터가 어떤 클래스 별로 수치를 비교하는 것이라면 교차검증 자체도 그룹핑으로 접근 할 수 있는것

In [7]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

In [13]:
scores_loo = cross_val_score(dt_clf,df_tt,df['survived'],scoring='accuracy',cv=loo)
print(len(scores_loo))
print(scores_loo.mean())

891
0.6745230078563412


In [15]:
## cross val 응용하기 
model = [DecisionTreeClassifier(),RandomForestClassifier(),LogisticRegression()] 
name = ['DT','RF','LR']

for model ,name in zip(model, name):
    print('******************사용한 알고리즘',name,'************************')
    for score in ['accuracy','precision','recall','f1']:
        print(score)
        print('--')
        print(cross_val_score(model, df_tt, df['survived'], scoring=score,cv=10))

******************사용한 알고리즘 DT ************************
accuracy
--
[0.67777778 0.61797753 0.64044944 0.66292135 0.6741573  0.71910112
 0.60674157 0.78651685 0.71910112 0.75280899]
precision
--
[0.63636364 0.5        0.54545455 0.59090909 0.6        0.63636364
 0.5        0.74193548 0.65517241 0.76      ]
recall
--
[0.4        0.35294118 0.35294118 0.38235294 0.44117647 0.61764706
 0.32352941 0.67647059 0.55882353 0.54285714]
f1
--
[0.49122807 0.4137931  0.42857143 0.46428571 0.50847458 0.62686567
 0.39285714 0.70769231 0.6031746  0.63333333]
******************사용한 알고리즘 RF ************************
accuracy
--
[0.68888889 0.58426966 0.68539326 0.68539326 0.69662921 0.68539326
 0.66292135 0.76404494 0.73033708 0.76404494]
precision
--
[0.69230769 0.48       0.6        0.6        0.61538462 0.61111111
 0.56       0.70588235 0.66666667 0.7       ]
recall
--
[0.45714286 0.35294118 0.44117647 0.5        0.5        0.67647059
 0.41176471 0.67647059 0.58823529 0.57142857]
f1
--
[0.56666667 0.406

### 분류 평가지표
- 정확도 : 맞춘수(TP+FN) / 전체 데이터 건수

## 필수과제 1 
- weighted 구글링해서 내용 설명과 이것에 대한 코드
- 라벨의 개수에 비례하게 가중치를 줘서 평균 값
- 예를들면 위에 예제에서 (0.5)*(0.2) + (0.1)*(0.2) + (0.5)*(0.3) + (0.5)*(0.3) 

## 필수과제2 
- leaveOneOut말고 다른 교차검증 패키지 사용해보기 
- 개념설명과 실제 적용해서 나온 결과 값을 비교하기 
- e.g ) 타이타닉 데이터로 한다면 기존에 교차검증과 새롭게 진행한 교차검증의 차이